# Baseline Solution - Monte Carlo Dropout

## This notebook documents the baseline solution for ADC 2022. 

## Overview
Our challenge is to provide 1. an error estimates (for Light Track) and/or 2. a conditional probability distribution (for Regular Track) for each target (6 in total) given an observation from the Ariel Space Telescope. The light track encourages a natural progression to the regular track. Participants are welcomed to join either or both tracks as they see fit. 

Depending on the information content of the observation and the associated observation noise (which is a function of the instrument and the planetary system), the resultant error bounds on each target and their joint conditional distribution will be different.

There are many directions you can take to tackle the problem on hand. We would like to get you started with our baseline solution. 

Spectroscopic data alone are usually informative enough to provide a reasonable estiamte on the targets. After all, the trough and peaks in the spectra encoded information about the relative abundance of each gaseous species (see [Yip et al.](https://iopscience.iop.org/article/10.3847/1538-3881/ac1744>) ). The supplementary information also helps to better constrain some of the phyiscal quantities (see our discussion [here](https://www.ariel-datachallenge.space/ML/documentation/about) if you want to learn about the underlying physics :) , but I shall leave that to you. 

The baseline solution trains a CNN to output a deterministic estimate for each atmospheric target. At inference time, the network is made to produce probabilistic output by activating the dropout layers in the network (Monte Carlo Dropout, [Gal et al. 2016](https://arxiv.org/abs/1506.02142)). 

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import h5py
import os
import matplotlib.pyplot as plt
from keras.layers import Dense, Reshape, Input, Concatenate, BatchNormalization, Dropout, Conv1D,Flatten,MaxPooling1D
from keras.models import Model
from tqdm import tqdm
from helper import *
from preprocessing import *
from submit_format import *
from metric_light_track import *
from metric_regular_track import *

from MCDropout import MC_Convtrainer

### Fix seed


In [ ]:
np.random.seed(42)
keras.utils.set_random_seed(42)

### Constants

In [ ]:
RJUP = 69911000
MJUP = 1.898e27
RSOL = 696340000

## Read training data

In [ ]:
training_path = '/TrainingData/'


In [ ]:
test_path = '/TestData/'


In [ ]:
training_GT_path = os.path.join(training_path, 'Ground Truth Package')

In [ ]:
spectral_training_data = h5py.File(os.path.join(training_path,'SpectralData.hdf5'),"r")
aux_training_data = pd.read_csv(os.path.join(training_path,'AuxillaryTable.csv'))
soft_label_data = pd.read_csv(os.path.join(training_GT_path, 'FM_Parameter_Table.csv'))


## Extract Spectral data
Spectral data lives in a h5py format, which is useful for navigating different cases, but their format makes it difficult to bulk manage them. The helper function helps to transform the h5py file into a matrix of size N x 52 x 4
where N is the number of training examples, 52 is the number of wavelength channels and 4 is the observation data

In [ ]:
spec_matrix = to_observed_matrix(spectral_training_data,aux_training_data)
print("spectral matrix shape:", spec_matrix.shape)

# Visualising a single spectrum

In [ ]:
def visualise_spectrum(spectrum):
    fig = plt.figure(figsize=(10,6))
    plt.errorbar(x=spectrum[:,0], y= spectrum[:,1], yerr=spectrum[:,2] )
    ## usually we visualise it in log-scale
    plt.xscale('log')
    plt.show()

In [ ]:
visualise_spectrum(spec_matrix[1])

In [ ]:
## lets look at another one
visualise_spectrum(spec_matrix[2])

it is immediately apparent that the average transit depth between two spectra can change for an order of magnitude difference. The magnitude of the uncertainty can also change accordingly ( and is a function of the planetary system, brightness of the host star and instrument response function). 

## Pre-processing

### Settings

In [ ]:
repeat = 5
threshold = 0.8 ## for train valid split.
N = 5000 # train on the first 5000 data instances, remember only the first 26k examples are labelled, the rest are unlabelled!

We can safely discard wlgrid (wavelength grid) and wlwidth (width of wavelength) since they are unchanged in the dataset

### Extract Spectrum

In [ ]:
## extract the noise
noise = spec_matrix[:N,:,2]
## We will incorporate the noise profile into the observed spectrum by treating the noise as Gaussian noise.
spectra = spec_matrix[:N,:,1]
wl_channels = len(spec_matrix[0,:,0])
global_mean = np.mean(spectra)
global_std = np.std(spectra)


### Add additional features - radius of the star and the planet
Most of the time we know something about the planetary system before we even attempt to make an observation (we cant just point randomly with a multi-million euros instrument!). Some of these auxillary data may be useful for retrieval, here we are only using the radius of the star and the planet.

In [ ]:
## add Rstar and Rplanet
radii = aux_training_data[['star_radius_m', 'planet_radius_m']]
## we would prefer to use Rsol and Rjup 
radii['star_radius'] = radii['star_radius_m']/RSOL
radii['planet_radius'] = radii['planet_radius_m']/RJUP
radii = radii.drop(['star_radius_m', 'planet_radius_m'],axis=1)
radii = radii.iloc[:N, :]
mean_radii = radii.mean()
stdev = radii.std()

### get target

In [ ]:
target_labels = ['planet_temp','log_H2O','log_CO2','log_CH4','log_CO','log_NH3']
targets = soft_label_data.iloc[:N][target_labels]
num_targets = targets.shape[1]
targets_mean = targets.mean()
targets_std = targets.std()

## Train/valid Split

In [ ]:
ind = np.random.rand(len(spectra)) < threshold
training_spectra, training_radii,training_targets, training_noise = spectra[ind],radii[ind],targets[ind], noise[ind]
valid_spectra, valid_radii, valid_targets = spectra[~ind],radii[~ind],targets[~ind]


## Augment the dataset with noise (create multiple instances)
Observational noise from Ariel forms an important part of the challenge, any model must recognise that the observation are not absolute measurement and could vary (according to the uncertainty), as that will affect the uncertainty associated with our atmospheric targets. Here we try to incorporate these information by augmenting the data with the mean noise.

In [ ]:
aug_spectra = augment_data_with_noise(training_spectra, training_noise, repeat)
aug_radii = np.tile(training_radii.values,(repeat,1))
aug_targets = np.tile(training_targets.values,(repeat,1))

### Standardise the data

### spectra

In [ ]:
## standardise the input using global mean and stdev
std_aug_spectra = standardise(aug_spectra, global_mean, global_std)
std_aug_spectra = std_aug_spectra.reshape(-1, wl_channels)
std_valid_spectra = standardise(valid_spectra, global_mean, global_std)
std_valid_spectra = std_valid_spectra.reshape(-1, wl_channels)

### radius

In [ ]:
## standardise
std_aug_radii= standardise(aug_radii, mean_radii.values.reshape(1,-1), stdev.values.reshape(1,-1))
std_valid_radii= standardise(valid_radii, mean_radii, stdev)


### target
We are asking the model to provide estimates for 6 atmospheric targets. In this example will be performing a supervised learning task. 

In [ ]:
std_aug_targets = standardise(aug_targets, targets_mean.values.reshape(1,-1), targets_std.values.reshape(1,-1))
std_valid_targets = standardise(valid_targets, targets_mean, targets_std)

# Setup network


### hyperparameter settings


In [ ]:
batch_size= 32
lr= 1e-3
epochs = 10
filters = [32,64,128]
dropout = 0.1
# number of examples to generate in evaluation time (5000 is max for this competition)
N_samples = 5000

We followed [Yip et al.](https://iopscience.iop.org/article/10.3847/1538-3881/ac1744>) and adopted a simple CNN structure and loss function. 


In [ ]:
model = MC_Convtrainer(wl_channels,num_targets,dropout,filters)

In [ ]:
model.summary()

### Compile model and Train!

In [ ]:
## compile model and run
model.compile(
    optimizer=keras.optimizers.Adam(lr),
    loss='mse',)
model.fit([std_aug_spectra,std_aug_radii], 
          std_aug_targets, 
          validation_data=([std_valid_spectra, std_valid_radii],std_valid_targets),
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle=False,)


### evaluate validation data

In [ ]:
## select the corresponding GT for the validation data, and in the correct order.
index= np.arange(len(ind))
valid_index = index[~ind]

In [ ]:
##generate trace data using dropout.

In [ ]:
instances = N_samples
y_pred_valid = np.zeros((instances, len(std_valid_spectra), num_targets ))
for i in tqdm(range(instances)):
    y_pred = model.predict([std_valid_spectra,std_valid_radii])
    y_pred_valid[i] += y_pred

In [ ]:
y_pred_valid = y_pred_valid.reshape(-1,num_targets)

In [ ]:
## transform them back to original space

In [ ]:
y_pred_valid_org = transform_back(y_pred_valid,targets_mean[None, ...], targets_std[None, ...])
y_pred_valid_org = y_pred_valid_org.reshape(instances, len(std_valid_spectra), num_targets)
y_pred_valid_org = np.swapaxes(y_pred_valid_org, 1,0)

### Evaluate - light track

In [ ]:
### load the ground truth
GT_Quartiles_path = os.path.join(training_GT_path, 'QuartilesTable.csv')

In [ ]:
all_qs = load_Quartile_Table(GT_Quartiles_path)

In [ ]:
valid_GT_Quartiles = all_qs[valid_index]
valid_GT_Quartiles = np.swapaxes(valid_GT_Quartiles, 1,0)

In [ ]:
## extract relevant quariltes from trace data

In [ ]:
q1_pred_valid, q2_pred_valid, q3_pred_valid = np.quantile(y_pred_valid_org, [0.16,0.5,0.84],axis=1)

In [ ]:
# put them into correct format
valid_q_pred = np.concatenate([q1_pred_valid[None,...], q2_pred_valid[None,...], q3_pred_valid[None,...]],axis=0)

In [ ]:
# calculate!
light_track_metric(valid_GT_Quartiles, valid_q_pred, k =1000)

### Evaluate - regular track

In [ ]:
## read trace and quartiles table 
GT_trace_path = os.path.join(training_GT_path, 'Tracedata.hdf5')

In [ ]:
# assuming each distribution produce the same number of trace (N_samples)
trace1_matrix = y_pred_valid_org
# assuming uniform weight, and the weights must sum to 1
trace1_weights_matrix = np.ones((trace1_matrix.shape[0], trace1_matrix.shape[1]))/trace1_matrix.shape[1] 

calculate the score. Note here that the GT trace data argument requires only the path to the tracedata.hdf5 file. It will open on its own.

In [ ]:
batch_calculate(trace1_matrix, trace1_weights_matrix, GT_trace_path, id_order = valid_index)

## Generate prediction for test data

In [ ]:
spec_test_data = h5py.File(os.path.join(test_path,'SpectralData.hdf5'),"r")
aux_test_data = pd.read_csv(os.path.join(test_path,'AuxillaryTable.csv'))

In [ ]:
test_spec_matrix = to_observed_matrix(spec_test_data,aux_test_data )


### same pre-processing as before...

In [ ]:
std_test_spectra = standardise(test_spec_matrix[:,:,1], global_mean, global_std)


In [ ]:
test_radii = aux_test_data[['star_radius_m', 'planet_radius_m']]
## we would prefer to use Rsol and Rjup 
test_radii['star_radius'] = test_radii['star_radius_m']/RSOL
test_radii['planet_radius'] = test_radii['planet_radius_m']/RJUP
test_radii = test_radii.drop(['star_radius_m', 'planet_radius_m'],axis=1)

std_test_radii= standardise(test_radii, mean_radii, stdev)


## Predict

In [ ]:
instances = N_samples
y_pred_distribution = np.zeros((instances, len(std_test_spectra), num_targets ))
for i in tqdm(range(instances)):
    y_pred = model.predict([std_test_spectra,test_radii])
    y_pred_distribution[i] += y_pred


In [ ]:
y_pred_distribution = y_pred_distribution.reshape(-1,num_targets)

In [ ]:
## project back to original space
y_pred_org = transform_back(y_pred_distribution,targets_mean[None, ...], targets_std[None, ...])

In [ ]:
y_pred_org = y_pred_org.reshape(instances, len(std_test_spectra), num_targets)
y_pred_org = np.swapaxes(y_pred_org, 1,0)

## Package output into desired format
We follow specific formats in the competition, to help make the process as painless as possible, we have included a few helper functions to make sure you have the right format in place for the submission. 

### Light Track

In [ ]:
# extract quartiles estimate for 16th, 50th and 84th percentile.
all_q1_pred, all_q2_pred, all_q3_pred = np.quantile(y_pred_org, [0.16,0.5,0.84],axis=1)

In [ ]:
LT_submission = to_light_track_format(all_q1_pred, all_q2_pred, all_q3_pred)

### Regular Track

In [ ]:
tracedata = y_pred_org
# weight takes into account the importance of each point in the tracedata. 
# Currently they are all equally weighted and thus I have created a dummy array that sums the contribution to 1
weight = np.ones((y_pred_org.shape[0],y_pred_org.shape[1]))/np.sum(np.ones(y_pred_org.shape[1]) )

RT_submission = to_regular_track_format(y_pred_org, 
                                        weight, 
                                        name="RT_submission.hdf5")

## check!

In [ ]:
LT_submission.head()

In [ ]:
f = h5py.File("RT_submission.hdf5",'r')

In [ ]:
f['Planet_0']['tracedata'][:]

## Future work

There are different direction to take from here on, let us summarise the shortcoming of this model:
- The data preprocessing is quite simplitic and could have invested more efforts.
- we have only used 5000 data points, instead of the full dataset
- we didnt train the model with results from the retrieval (QuartilesTable.csv for Light Track and Tracedata.hdf5 for Regular Track), which are the GT for this competition.
- The conditional distribution from MCDropout is very restricted and Gaussian-like
- So far we havent considered the atmospheric targets as a joint distribution
- We have only used stellar radius and planet radius from the auxillary information
- We have not done any hyperparameter tuning 
- the train test split here is not clean, as in, we split the data after we have augmented the data, which results in information leakage to the validation data. There is no leakage to the test data though.